In [2]:
import os

import numpy as np
from keras import Sequential, Model
from keras.layers import Dense
from tensorflow.python.keras.optimizer_v2.adam import Adam

from src.aes.sbox import s_box
from src.data.loaders.ascad import ASCADData
from src.dlla.hw import encode, dlla_hw
from src.dlla.wegener import make_mlp_wegener, binomial_test
from src.pollution.clock_jitter import clock_jitter
from src.pollution.gaussian_noise import gaussian_noise
from src.pollution.random_delay import random_delay
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW
from src.trace_set.transform import reduce_fixed_fixed, fixed_fixed
from src.trace_set.window import get_windows, extract_traces
from src.tvla.cri import tvla_cri
from src.tvla.tvla import Group

In [3]:
DB = Database.ascad
RAW_DATA = ASCADData.raw()

LIMIT_PROF = None
LIMIT_ATT = None

TARGET_ROUND = 0
TARGET_BYTE = 2 # ASCAD: 0 for unprotected, 2 for masked byte.

In [4]:
RAW_TRACES = RAW_DATA['traces']
TRACE_SET = TraceSetHW(DB)

SAMPLE_TRACE = TRACE_SET.profile()[0][0]
WINDOW, WINDOW_CXT = get_windows(RAW_TRACES, SAMPLE_TRACE)

WINDOW = WINDOW[0] + 1150, WINDOW[0] + 1450

In [5]:
# X_CXT = extract_traces(RAW_TRACES, WINDOW_CXT)[:-1]

In [6]:
# PROFILING_MASK = np.ones(len(X_CXT), dtype=bool)
# PROFILING_MASK[2::3] = 0
#
# PROF_PLAIN, PROF_KEY = TRACE_SET.profile_meta()
# ATT_PLAIN, ATT_KEY = TRACE_SET.attack_meta()

In [7]:
def state_byte_round_0(plain, key):
    return s_box[plain ^ key]

# Y_PROF = state_byte_round_0(PROF_PLAIN, PROF_KEY)
# Y_ATT = state_byte_round_0(ATT_PLAIN, ATT_KEY)
#
# X_PROF_CXT = X_CXT[PROFILING_MASK]
# X_ATT_CXT = X_CXT[~PROFILING_MASK]
#
# X_PROF, _ = TRACE_SET.profile()
# X_ATT, _ = TRACE_SET.attack()

# X_PROF_CXT.shape, PROF_KEY.shape

In [8]:
JITTER_PARAMS = np.arange(0, .02, .01)

# def apply_jitter(params):
#     for param in params:
#         pollution = Pollution(PollutionType.jitter, param)
#         out = TraceSetHW(DB, pollution)
#
#         if not os.path.exists(out.path):
#             x = clock_jitter(X_PROF_CXT, WINDOW, param)
#             x_att = clock_jitter(X_ATT_CXT, WINDOW, param)
#
#             out.create(x, Y_PROF, x_att, Y_ATT)
#
#             x2, y2 = fixed_fixed(x, Y_PROF)
#             a, b = x2[~y2], x2[y2]
#             order = 2
#             _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
#             print(f"Clock jitter ({param}): min-p: ({min(tvla_p)}).")

# apply_jitter(JITTER_PARAMS)

In [9]:
DELAY_PARAMS = np.arange(0, .02, .01)
A = 5
B = 3
DELAY_AMP = 10

# def apply_delay(params):
#     for param in params:
#         pollution = Pollution(PollutionType.delay, param)
#         out = TraceSetHW(DB, pollution)
#
#         if not os.path.exists(out.path):
#             x = random_delay(X_PROF, A, B, DELAY_AMP, param)
#             x_att = random_delay(X_ATT, A, B, DELAY_AMP, param)
#
#             out.create(x, Y_PROF, x_att, Y_ATT)
#
#             x2, y2 = fixed_fixed(x, Y_PROF)
#             a, b = x2[~y2], x2[y2]
#             order = 2
#             _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
#             print(f"Random delay ({param}): min-p: ({min(tvla_p)}).")

# apply_delay(DELAY_PARAMS)

In [10]:
GAUSS_PARAMS = np.arange(0, 22, 1)

### TVLA vs. DL-LA

In [11]:
ORDER = 2

def store_results(database: Database, method: str, pollution: Pollution, p):
    file_name = f"results_{database.name}.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{pollution.type.name};{pollution.parameter};{p}\n")

In [13]:
def prepare_traces_dl(x, y, x_att, y_att):
    """
    Normalizes the traces, one-hot encodes the labels.
    Returns profiling traces, labels and attack traces, labels.
    """
    prof_mean, prof_std = x.mean(axis=0), x.std(axis=0)
    norm_x = (x - prof_mean) / prof_std
    norm_x_att = (x_att - prof_mean) / prof_std

    return norm_x, encode(y), norm_x_att, encode(y_att)


def build_mlp(x, y, params):
    mdl = Sequential()
    mdl.add(Dense(100, activation=params['activation'], input_shape=(x.shape[1],)))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(9, activation='softmax'))

    mdl.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['accuracy'])

    out = mdl.fit(x, y, shuffle=True, batch_size=params['batch_size'], epochs=params['epochs'], verbose=False)

    return out, mdl


def make_mlp(x, y):
    return build_mlp(x, y, {
        'activation': 'relu',
        'optimizer': Adam(learning_rate=0.001),
        'losses': 'categorical_crossentropy',
        'batch_size': 150,
        'epochs': 5
    })[1]

def wegener_p(mdl: Model, x_att: np.ndarray, y_att: np.ndarray):
    predictions = mdl.predict(x_att).argmax(axis=1)
    labels = y_att.argmax(axis=1)

    correct = np.sum(predictions == labels)
    total = len(predictions)
    # print(total, correct)

    return binomial_test(total, correct)

def la_benchmark(db: Database, pollution_type: PollutionType, params):
    for param in params:
        pollution = Pollution(pollution_type, param)
        print("Load traces         ", end="\r")
        trace_set = TraceSetHW(db, pollution, (LIMIT_PROF, LIMIT_ATT))

        if os.path.exists(trace_set.path):
            print("Prepare traces (1/2)", end="\r")
            x9, y9, x9_att, y9_att = prepare_traces_dl(*trace_set.profile(), *trace_set.attack())
            print("Prepare traces (2/2)", end="\r")
            (x2, y2), (x2_att, y2_att) = reduce_fixed_fixed(x9, y9), reduce_fixed_fixed(x9_att, y9_att)

            print("Make model (1/2)    ", end="\r")
            mdl9 = make_mlp(x9, y9)
            print("Make model (2/2)    ", end="\r")
            mdl2 = make_mlp_wegener(x2, y2, False)

            print("TVLA                ", end="\r")
            tvla_ps = np.min(tvla_cri(*fixed_fixed(*trace_set.profile()), ORDER), axis=1)

            print("Predict             ", end="\r")
            dlla9_p = dlla_hw(mdl9, x9_att, y9_att)
            dlla2_p = wegener_p(mdl2, x2_att, y2_att)

            print(f"{pollution_type} ({param}). TVLA ({tvla_ps}). DLLA9 ({dlla9_p}). DLLA2 ({dlla2_p})", end="\r")
            # print(f"{pollution_type} ({param}). TVLA ({tvla_ps})", end="\r")

            for order, p in enumerate(tvla_ps):
                if order > 0:
                    store_results(db, f"cri_tvla_{order}", pollution, p)

            store_results(db, "dlla9", pollution, dlla9_p)
            store_results(db, "dlla2", pollution, dlla2_p)

        print()

while True:
    # la_benchmark(DB, PollutionType.jitter, JITTER_PARAMS)
    # la_benchmark(DB, PollutionType.delay, DELAY_PARAMS)
    la_benchmark(DB, PollutionType.gauss, GAUSS_PARAMS)


PollutionType.gauss (0). TVLA ([1.00000000e+00 4.67799139e-03 7.24705065e-11]). DLLA9 (0.0). DLLA2 (0.0)
PollutionType.gauss (1). TVLA ([1.00000000e+00 3.67444633e-02 7.86931526e-12]). DLLA9 (0.0). DLLA2 (0.0)


KeyboardInterrupt: 

In [14]:
la_benchmark(DB, PollutionType.gauss, [40])

PollutionType.gauss (40). TVLA ([1.         0.03040189 0.03010203]). DLLA9 (0.024706569401522823). DLLA2 (0.5706876276054733)
